# **This code extracts data from portfolio file (ideally in pdf format) and latest trading data from Dhaka Stock Exchange Website, then portrays the current position of the portfolio. The scenario is updated every 30 seconds for the sake of continuous monitoring. **





Disclaimer

    1. You need to upload the portfolio pdf file from the "Files" section. The preferred name is "10.pdf", if your file has a different name, rename it or make a change in the code. 
    2. The broker provided pdf file might not suit the code, in that case you may provide the initial data in .xlsx file, that would require minor change in the code obviously. 
    3. The gain/loss is calculated considering Latest Trading Price during the trading hours and Closing Price after trading hours.


In [ ]:
# check if bdshare pypdf and tabula are installed if not install
try:
    from bdshare import get_current_trade_data
except:
    !pip install bdshare
    from bdshare import get_current_trade_data
 
try:
    import tabula
except:
    !pip install tabula-py
    import tabula
 
# import required modules
import pandas as pd
import numpy as np
 
# set the number format
pd.options.display.float_format = '{:.2f}'.format
 
# set pandas display width
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 14)
pd.set_option("display.width", 1000)
 
# get the data
df_org = tabula.read_pdf("10.pdf", multiple_tables=True, pages="all")[0]
port = df_org.iloc[1:-2, :]
req_cols = ["Company", "Total", "Average", "Total Cost"]
port = port[req_cols]
port["Total Cost"] = port["Total Cost"].str.replace(",", "")
cols = ["Total", "Average", "Total Cost"]
port[cols] = port[cols].apply(pd.to_numeric, errors="coerce")
port.columns = ["TRADING.CODE", "Total Quantity", "Average Cost", "Total Cost"]
 
# Get the instruments under portfolio
instruments = port["TRADING.CODE"]
instruments = instruments.tolist()
 
# function for extracting the data
def dse_current(instruments):
    df = []
 
    for instruments in instruments:
        stock_data = get_current_trade_data(instruments)
        df.append(stock_data)
 
    df = pd.concat(df)
    df = df.sort_values(by=["symbol"])
    df = df[
        ["symbol", "ltp", "high", "low", "close", "ycp", "trade", "value", "volume"]
    ]
    df = df.reset_index()
    df = df.drop(columns=["index"])
    df.columns = [
        "TRADING.CODE",
        "LTP",
        "HIGH",
        "LOW",
        "CLOSEP",
        "YCP",
        "TRADE",
        "VALUE",
        "VOLUME",
    ]
    cols = df.columns.drop(["TRADING.CODE"])
    df[cols] = df[cols].apply(pd.to_numeric, errors="coerce")
 
    # merge the data with uploaded portfolio
    df = pd.merge(port, df, on="TRADING.CODE")
 
    # get current gain/loss
    if (df["CLOSEP"].sum()==0):
      df["Current_Val"] = np.where(df["LTP"] > 0, df["Total Quantity"] * df["LTP"], df["Total Quantity"] * df["YCP"])
    else: 
      df["Current_Val"] = df["Total Quantity"] * df["CLOSEP"] 
 
    df["Gain/Loss"] = df["Current_Val"] - df["Total Cost"]
 
    # generate the summary table
    total_quan = df["Total Quantity"].sum()
    total_cost = df["Total Cost"].sum()
    mkt_val = df["Current_Val"].sum()
    gain = df["Gain/Loss"].sum()
    summary = [total_quan, total_cost, mkt_val, gain]
    summary_tab = pd.DataFrame(np.array(summary).reshape(1, 4), columns = ["Total Shares", "Total Cost", "Total Market Value", "Total Gain/Loss"])
 
    return (df, summary_tab)

Got stderr: Sep 30, 2020 8:47:41 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Sep 30, 2020 8:47:41 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [ ]:
# repeat the function after the given interval
from datetime import datetime
import time
from IPython.display import clear_output
from pytz import timezone

bd = timezone("Asia/Dhaka")

def periodic_position(interval):
    while True:
        current_pos = dse_current(instruments)[0]
        summary_tab = dse_current(instruments)[1]
        clear_output(wait=True)
        print("As of:")
        print(datetime.now(bd).strftime("%H:%M:%S.%f - %b %d %Y"))
        print("---")
        print(current_pos)
        print("---")
        #print("Current Total Gain/Loss is: {:2f}".format(gain))
        print(summary_tab)
        time.sleep(interval)

In [ ]:
# now keep running the loop
periodic_position(30)

As of:
15:22:19.545299 - Sep 30 2020
---
  TRADING.CODE  Total Quantity  Average Cost  Total Cost    LTP   HIGH    LOW  CLOSEP    YCP  TRADE  VALUE   VOLUME  Current_Val   Gain/Loss
0     BANKASIA          800405         18.60 14889729.66  18.50  18.50  18.20   18.40  18.30    170   9.23   504326  14727452.00  -162277.66
1          DBH           25000         94.29  2357227.50  92.60  93.00  92.60   92.60  92.60    692  77.48   836411   2315000.00   -42227.50
2          EBL          191080         36.82  7034935.88  36.50  36.50  35.90   36.20  35.90    120   5.39   149263   6917096.00  -117839.88
3   JAMUNABANK         3017266         17.98 54247333.93  17.50  17.90  17.40   17.50  17.60    138  12.43   711166  52802155.00 -1445178.93
4   MERCANBANK         2542642         12.94 32909199.86  12.50  12.70  12.40   12.50  12.60    281  14.05  1119658  31783025.00 -1126174.86
5      NCCBANK         4413769         13.96 61627468.32  13.50  13.70  13.30   13.40  13.50    149   6.08   4532